In [ ]:
# default_exp resnetx2

In [ ]:
#export
from wong.imports import *
from wong.core import *
from wong.config import cfg, assert_cfg


In [ ]:
from fastcore.all import *  # test_eq

# ResNet4

In [ ]:
#export
class Init4(nn.Module):
    "Init block of folded ResNet"
    def __init__(self, Unit:nn.Module, ni:int, fold:int, **kwargs):
        super(Init4, self).__init__()
        unit1 = Unit(ni, stride=1, **kwargs)
        unit2 = Unit(ni, stride=1, **kwargs)
        unit3 = Unit(ni, stride=1, **kwargs)
        
    def forward(self, x):
        x1 = unit1(x) + x
        x2 = unit2(x1) + x1
        x3 = unit3(x2) + x2
        return x3, x2, x1, x

# ResNetX2
> a folded ResNet

Aggregate to enough units for folded net:
1. Unit : unit operator
2. ni : number of input channels for `Unit`
3. fold : folding length
4. stride : across stage or not
5. **kwargs : arguments to `Unit`



In [ ]:
#export
class InitBlock(nn.Module):
    "Init block of folded ResNet"
    def __init__(self, Unit:nn.Module, ni:int, fold:int, stride:int=1, **kwargs):
        super(InitBlock, self).__init__()
        self.ni, self.fold = ni, fold
        units = []
        for i in range(fold-1):
            units += [Unit(ni, stride=stride, **kwargs)]
        self.units = nn.ModuleList(units)
        
    def forward(self, x):
        xs = [x]
        for i in range(self.fold-1):
            xs += [xs[i] + self.units[i](xs[i])]
        xs.reverse()
        return xs

In [ ]:
m = InitBlock(mbconv, 16, 4, stride=1, nh=32)
x = torch.randn(2,16,32,32)
xs = m(x)
len(xs), xs[0].shape

(4, torch.Size([2, 16, 32, 32]))

In [ ]:
#export
class FoldBlock(nn.Module):
    "Basic block of folded ResNet"
    def __init__(self, Unit:nn.Module, ni:int, fold:int, stride:int=1, **kwargs):
        super(FoldBlock, self).__init__()
        self.ni, self.fold, self.stride = ni, fold, stride
        units = []
        for i in range(fold-1):
            units += [Unit(ni, stride=1, **kwargs)]
        self.units = nn.ModuleList(units)
        
    def forward(self, *xs):
        xs = list(xs)
        for i in range(self.fold-1):
            xs[i+1] = xs[i+1] + self.units[i](xs[i])
        xs.reverse()
        return xs

In [ ]:
m = FoldBlock(mbconv, 16, 4, nh=32)

In [ ]:
xs2 = m(*xs)

In [ ]:
len(xs2),xs2[0].shape,xs2[1].shape,xs2[2].shape,xs2[3].shape

(4,
 torch.Size([2, 16, 32, 32]),
 torch.Size([2, 16, 32, 32]),
 torch.Size([2, 16, 32, 32]),
 torch.Size([2, 16, 32, 32]))

In [ ]:
#export
class ExpandBlock(nn.Module):
    "Expand block of folded ResNet"
    def __init__(self, Unit:nn.Module, ni:int, fold1:int, fold2:int, stride:int=1, **kwargs):
        super(ExpandBlock, self).__init__()
        self.ni, self.fold1, self.fold2, self.stride = ni, fold1, fold2, stride
        units = []
        for i in range(fold2 - fold1):
            units += [Unit(ni, stride=1, **kwargs)]
        self.units = nn.ModuleList(units)
        if stride == 2:
            self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
    def forward(self, *xs):
        xs = list(xs)
        if self.stride == 2:
            for i in range(len(xs)):
                xs[i] = self.pool(xs[i])
        xs.reverse()
        for i in range(self.fold2 - self.fold1):
            xs.append(self.units[i](xs[-1]) + xs[-1])
        xs.reverse()
        return xs

In [ ]:
m = ExpandBlock(mbconv, 16, fold1=4, fold2=8, stride=2, nh=32)

In [ ]:
xs2 = m(*xs)

In [ ]:
len(xs2),xs2[0].shape,xs2[1].shape,xs2[2].shape,xs2[3].shape

(8,
 torch.Size([2, 16, 16, 16]),
 torch.Size([2, 16, 16, 16]),
 torch.Size([2, 16, 16, 16]),
 torch.Size([2, 16, 16, 16]))

In [ ]:
#export
class ResNetX2(nn.Module):
    "A folded resnet."
    def __init__(self, Stem, Unit, fold:int, ni:int, num_nodes:tuple, exp:int=1,
                 bottle_scale:int=1, first_downsample:bool=False,
                 c_in:int=3, c_out:int=10, **kwargs):
        super(ResNetX2, self).__init__()
        num_stages = len(num_nodes)
        nh = ni * bottle_scale
        strides = [1 if i==0 and not first_downsample else 2 for i in range(num_stages)]
        folds = [1] + [fold*exp**i for i in range(num_stages)]
        
        self.stem = Stem(c_in, no=ni) # , deep_stem
        #self.init = InitBlock(Unit, ni, fold, nh=nh)
        
        units = []
        idmappings = []
        cur = 1
        for i, (nu, stride) in enumerate(zip(num_nodes, strides)):
            for j in range(nu):
                if j == 0: # the first node(layer) of each stage
                    units += [ExpandBlock(Unit, ni, fold1 = folds[i], fold2=folds[i+1], stride=stride, nh=nh, **kwargs)]
                else:
                    units += [FoldBlock(Unit, ni, fold=folds[i+1], stride=1, nh=nh, **kwargs)]
                    
        self.units = nn.ModuleList(units)
        
        self.classifier = Classifier(ni*folds[-1], c_out) #*fold
        self.fold = fold
        self.num_nodes = num_nodes
        init_cnn(self)
        
    def forward(self, x):
        x = self.stem(x)
        xs = [x] #self.init(x)
        for unit in self.units:
            xs = unit(*xs)
        x = torch.cat(xs,1)  #xs[0] 

        x = self.classifier(x)
        return x
        

In [ ]:
num_nodes = [8]*4
model = ResNetX2(Stem=conv_bn, Unit=mbconv, fold=4, ni=64, num_nodes=num_nodes, exp=2, bottle_scale=2)
num_params(model)

tensor(7710794)

In [ ]:
x = torch.randn(2,3,64,64)

In [ ]:
with torch.autograd.set_detect_anomaly(True):
    out = model(x)
    out.mean().backward()